In [1]:
import configparser
from datetime import datetime
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format



In [2]:
def create_spark_session():
    spark = SparkSession \
        .builder \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
        .getOrCreate()
    return spark

In [4]:
def process_song_data(spark, input_data, output_data):
    # get filepath to song data file
    song_data = "C:/Udacity/DataEngineer/DataLakewithSpark/Project/data/song-data/song_data/A/A/A/TRAAAAW128F429D538.json"
    
    # read song data file
    df_songs_data = spark.read.json(song_data)
    df_songs_data.persist()

    # extract columns to create songs table
    songs_table = df_songs_data.select(['song_id', 'title', 'artist_id', 'year', 'duration']).collect()
    df_songs_data.createOrReplaceTempView("songs_table")

    # write  songs table to parquet files partitioned by year and artist
    songs_table.write.partitionBy("year","artist_id").mode('overwrite').parquet(os.path.join(output_data, 'songs'))

    # extract columns to create artists table
    artists_table = df_songs_data.select(['artist_id', 'name', 'location', 'lattitude', 'longitude']).collect() 
    
    # write artists table to parquet files
    artists_table.write.mode('overwrite').parquet(os.path.join(output_data, 'artists'))